# 1. Install and Import Dependencies

In [ ]:
!pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio===0.8.1 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
!pip install transformers requests beautifulsoup4 pandas numpy

In [1]:
import torch
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# 2. Instantiate Model

In [2]:
# Load pre-trained tokenizer and model for sentiment analysis
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

# 3. Encode and Calculate Sentiment

In [3]:
tokens = tokenizer.encode('It was good but could have been better. Great', return_tensors='pt')

In [4]:
result = model(tokens)

In [5]:
result.logits

tensor([[-2.8836, -1.3482,  1.4180,  2.0596,  0.5641]],
       grad_fn=<AddmmBackward>)

In [6]:
int(torch.argmax(result.logits))+1

4

# 4. Collect Reviews

In [3]:
# Fetch reviews from Yelp page
url = 'https://www.yelp.com/biz/social-brew-cafe-pyrmont'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Extract review paragraphs using BeautifulSoup and regex
comment_class_regex = re.compile('.*comment.*')
review_paragraphs = soup.find_all('p', {'class': comment_class_regex})
reviews = [paragraph.text for paragraph in review_paragraphs]

In [4]:
reviews

['Some of the best Milkshakes me and my daughter ever tasted. MMMMMM HMMMMMMMM.',
 "Six of us met here for breakfast before our walk to Manly. We were enjoying visiting with each other so much that I apologize for not taking any photos. We all enjoyed our food, as well as our coffee and tea drinks.We were greeted immediately by a friendly server asking if we would like to sit inside or out. We said we would like inside, but weren't exactly sure how many were joining us yet- at least 4. We were told this was no problem, the more the merrier. A few minutes later when 4 more joined our party and we explained to the server we had 6, he just quickly switched our table. I really enjoyed my serenity tea, just what I needed after a long flight in from Sfo that morning. Everyone else were more interested in the lattes for expresso drinks. All said they were hot and delicious. 2 of us ordered the avo on toast. So yummy with the beetroot... I will start adding this to mine now at home, and have f

# 5. Load Reviews into DataFrame and Score

In [5]:
# Create a DataFrame to store reviews
review_df = pd.DataFrame(np.array(reviews), columns=['review'])

In [7]:
review_df['review'].iloc[0]

'Some of the best Milkshakes me and my daughter ever tasted. MMMMMM HMMMMMMMM.'

In [8]:
# Function to calculate sentiment score
def calculate_sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    sentiment = int(torch.argmax(result.logits)) + 1
    return sentiment

In [9]:
calculate_sentiment_score(review_df['review'].iloc[1])

4

In [10]:
# Apply sentiment analysis to each review and add sentiment score to DataFrame
review_df['sentiment'] = review_df['review'].apply(lambda x: calculate_sentiment_score(x[:512]))

In [12]:
# Show the DataFrame
review_df

,review,sentiment
0,Some of the best Milkshakes me and my daughter...,5
1,Six of us met here for breakfast before our wa...,4
2,Great place with delicious food and friendly s...,5
3,Great food amazing coffee and tea. Short walk ...,5
4,It was ok. Had coffee with my friends. I'm new...,3
5,Ricotta hot cakes! These were so yummy. I ate ...,5
6,Great staff and food. Must try is the pan fri...,5
7,We came for brunch twice in our week-long visi...,4
8,I came to Social brew cafe for brunch while ex...,5
9,It was ok. The coffee wasn't the best but it w...,3
